Задание 1.
Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)
С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?
Повторим шаги из заданий 1 и 2, используя библиотеку nltk.


Задание 2.
Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.
С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?

Задание 3.
Какая из библиотек по вашему лучше отработала? Сравните качество полученных most_common NER и количество распознаных NER.


In [23]:
from tqdm import tqdm
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import numpy as np

In [24]:
tweets = pd.read_pickle('/content/drive/My Drive/clean.pkl')
tweets.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit can use cause they do n...,"[thanks, for, lyft, credit, can, use, cause, t...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in your,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time]","[model, love, take, time]","[model, love, take, time]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


In [33]:
import spacy
from spacy import displacy
import en_core_web_md
nlp = en_core_web_md.load()
nlp.max_length = 1000000

In [34]:
def ner_by_spacy(texts):

    entities = []
    labels = []
    for text in tqdm(texts):
        doc = nlp(text)
        for ent in doc.ents:
            entities.append(ent.text)
            labels.append(ent.label_)

    df = pd.DataFrame({'Entities': entities, 'Labels': labels})

    return df

In [60]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('maxent_ne_chunker') 
nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [36]:
df_spacy = ner_by_spacy(tweets['clean_tweet'])

100%|██████████| 49159/49159 [08:08<00:00, 100.71it/s]


In [40]:
df_gb = df_spacy.groupby(['Entities', 'Labels'])[['Entities']].count()

In [42]:
df_gb.columns = ['counts']
df_gb.reset_index(inplace=True)
df_gb.sort_values(by='counts', ascending=False).head(20)

,Entities,Labels,counts
16653,today,DATE,1401
16720,tomorrow,DATE,596
5668,friday,DATE,587
12086,one,CARDINAL,518
5386,first,ORDINAL,512
12245,orlando,GPE,468
15356,sunday,DATE,458
10817,morning,TIME,430
16739,tonight,TIME,419
15264,summer,DATE,402


In [44]:
df_gb_lb = df_spacy.groupby(['Labels'])[['Labels']].count()
df_gb_lb.columns = ['counts']
df_gb_lb.reset_index(inplace=True)
df_gb_lb.sort_values(by='counts', ascending=False).head()

,Labels,counts
13,PERSON,11715
1,DATE,10808
11,ORG,8165
4,GPE,5412
16,TIME,2289


In [45]:
train_df = tweets.loc[tweets.id < 31963]
test_df = tweets.loc[tweets.id > 31963]

In [46]:
train =  " ".join(train_df['clean_tweet'].values)
test =  " ".join(test_df['clean_tweet'].values)
train = train[:1000000]
test = test[:1000000]

In [51]:
train_person_dict = {}
train_org_dict = {}
test_person_dict = {}
test_org_dict = {}
def add_to_dict(dict_, word):
  if word in dict_.keys():
    dict_[word] += 1
  else:
    dict_[word] = 1
doc = nlp(train)
for entity in doc.ents:
  if entity.label_ == 'PERSON':
    add_to_dict(train_person_dict, entity.text)
  if entity.label_ == 'ORG':
    add_to_dict(train_org_dict, entity.text)
doc2 = nlp(test)
for entity in doc2.ents:
  if entity.label_ == 'PERSON':
    add_to_dict(test_person_dict, entity.text)
  if entity.label_ == 'ORG':
    add_to_dict(test_org_dict, entity.text)

In [52]:
train_person_df = pd.DataFrame.from_dict(train_person_dict, columns=['cnt'], orient='index')
train_org_df = pd.DataFrame.from_dict(train_org_dict, columns=['cnt'], orient='index')
train_person_df.sort_values(by='cnt', ascending=False, inplace=True)
train_org_df.sort_values(by='cnt', ascending=False, inplace=True)
test_person_df = pd.DataFrame.from_dict(test_person_dict, columns=['cnt'], orient='index')
test_org_df = pd.DataFrame.from_dict(test_org_dict, columns=['cnt'], orient='index')
test_person_df.sort_values(by='cnt', ascending=False, inplace=True)
test_org_df.sort_values(by='cnt', ascending=False, inplace=True)

In [53]:
train_person_df.head(20)

,cnt
obama,25
sta,25
suppo,20
deletetweets,20
hillary,17
bing bong bing bong,14
don,12
lebron,11
bjp,11
melancholy melancholymusic,11


In [54]:
test_person_df.head(20)

,cnt
suppo,28
obama,24
sta,22
deletetweets,13
christina grimmie,12
feminismisterrorism feminismmuktbharat,11
clinton,10
jo cox,9
don,9
bong bing bong,9


In [55]:
train_org_df.head(20)

,cnt
islam,19
nba,19
gop,19
sjw,15
wso,10
fed,10
cavs,9
buffalo simulation buffalo,8
google,7
eu,7


In [56]:
test_org_df.head(20)

,cnt
gop,20
nba,16
ht,14
islam,12
sjw,11
wa,10
buffalo simulation buffalo,8
fbi,8
facebook,8
congress,7


лишнее хоть и есть, но результат виден

In [57]:
df_nltk = ner_by_nltk(tweets['clean_tweet'])

100%|██████████| 49159/49159 [03:35<00:00, 227.92it/s]


In [62]:
text =  " ".join(tweets['clean_tweet'].values)
text = text[:1000000]

In [63]:
text = text.title()

In [64]:
ent_nltk_dict = {}
ent_nltk_types_dict = {}

for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))):
  if hasattr(chunk, 'label'):
    add_to_dict(ent_nltk_dict, (' '.join(c[0] for c in chunk)))
    add_to_dict(ent_nltk_types_dict, chunk.label())

ent_nltk_types_df = pd.DataFrame.from_dict(ent_nltk_types_dict, columns=['cnt'], orient='index')
ent_nltk_df = pd.DataFrame.from_dict(ent_nltk_dict, columns=['cnt'], orient='index')
ent_nltk_types_df.sort_values(by='cnt', ascending=False, inplace=True)
ent_nltk_df.sort_values(by='cnt', ascending=False, inplace=True)

In [65]:
ent_nltk_df.head(20)

,cnt
Will,100
New,78
Us,54
Delete Old,43
Orlando,42
Happy,40
Forward,36
Cold,33
Libtard,33
Father,32


In [66]:
ent_nltk_types_df.head(5)

,cnt
PERSON,12097
ORGANIZATION,5677
GPE,1711
FACILITY,442
LOCATION,28


In [67]:
train_nltk = train.title()
test_nltk = test.title()
train_person_dict = {}
train_org_dict = {}

for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(train_nltk))):
  if hasattr(chunk, 'label'):
    if chunk.label() == 'PERSON':
      add_to_dict(train_person_dict, (' '.join(c[0] for c in chunk)))
    if chunk.label() == 'ORGANIZATION':
      add_to_dict(train_org_dict, (' '.join(c[0] for c in chunk)))
train_person_df = pd.DataFrame.from_dict(train_person_dict, columns=['cnt'], orient='index')
train_org_df = pd.DataFrame.from_dict(train_org_dict, columns=['cnt'], orient='index')
train_person_df.sort_values(by='cnt', ascending=False, inplace=True)
train_org_df.sort_values(by='cnt', ascending=False, inplace=True)

In [71]:
test_person_dict = {}
test_org_dict = {}

for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(test_nltk))):
  if hasattr(chunk, 'label'):
    if chunk.label() == 'PERSON':
      add_to_dict(test_person_dict, (' '.join(c[0] for c in chunk)))
    if chunk.label() == 'ORGANIZATION':
      add_to_dict(test_org_dict, (' '.join(c[0] for c in chunk)))

In [75]:
test_person_df = pd.DataFrame.from_dict(test_person_dict, columns=['cnt'], orient='index')
test_org_df = pd.DataFrame.from_dict(test_org_dict, columns=['cnt'], orient='index')
test_person_df.sort_values(by='cnt', ascending=False, inplace=True)
test_org_df.sort_values(by='cnt', ascending=False, inplace=True)

In [74]:
train_person_df.head(20)

,cnt
Will,100
Delete Old,43
Forward,36
Happy,35
Libtard,33
Father,31
Grateful,27
Sad,25
Been,23
Gon,21


In [77]:
test_person_df.head(20)

,cnt
Will,95
Forward,40
Happy,40
Delete Old,34
Father,33
Ready,25
Libtard,24
Grateful,23
Good,21
Sad,19


In [69]:
train_org_df.head(20)

,cnt
Vicinity Of Their,25
Lot,21
Fathers,21
Vast Expanse Of Mountains,18
Well Deletetweets,18
House,16
All,15
Work Conference,14
Same,12
No,11


In [78]:
test_org_df.head(20)

,cnt
Vicinity Of Their,23
Vast Expanse Of Mountains,22
Lot,21
All,21
Happiness,20
Best,19
Same,16
House,15
Work Conference,15
Only,14


In [ ]:
nltk явно хуже, и настраивается сложнее и количество распознаных существенно меньше